In [24]:
import cmd
import re
import Bio
from Bio import Entrez
from Bio import Blast
from Bio import SeqIO
from Bio import AlignIO
import json
import textblob
from textblob import TextBlob
import nltk 
from nltk import word_tokenize
import pylab
from Bio import SeqIO
from Bio import PDB
import sys
eutils="http://eutils.ncbi.nlm.nih.gov"
Entrez.email = 'rjliangiogenesis@gmail.com' 


In [25]:
#set of functions for exploring biopython/getting protein CDS
import Bio
from Bio import Entrez
from Bio import Blast
from Bio import SeqIO
from Bio import AlignIO

def get_number_pubmed_queries(query): #input protein name and number of results returned, returns list of IDs for fetching
        Entrez.email = 'rjliangiogenesis@gmail.com' 
        handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax=1000,  
                            retmode='xml', 
                            term=query) 
        results = Entrez.read(handle) 
        return len(results["IdList"])

def get_IDs(query,number): #input protein name and number of results returned, returns list of IDs for fetching
        Entrez.email = 'rjliangiogenesis@gmail.com' 
        handle = Entrez.esearch(db='protein', 
                            sort='relevance', 
                            retmax=number,  
                            retmode='xml', 
                            term=query) 
        results = Entrez.read(handle) 
        return results["IdList"] 
    
def fetch_details(ids):   #input Ids from protsearchx, returns full documents including abstracts
        Entrez.email = 'rjliangiogenesis@gmail.com' 
        handle = Entrez.efetch(db='protein', 
                           rettype='gp', #9.6 of biopython documentation
                           retmode='xml', 
                           id=ids) 
        results = Entrez.read(handle) 
        return results

    
def isHuman(dictionary): #input list of dictionaries, returns boolean result
        if dictionary['GBSeq_source']=='Homo sapiens (human)':
            return True
        else:
            pass
    
def term_in_list(term, dictionaryList): #input string term + fetched document, return that term in each doc
        for dictionary in dictionaryList: 
            if term in dictionary: 
                sequences.append(dictionary[term]) 
        return sequences
    
def dictCreate(index,b,c): #creates a dictionary from 3 inputs. order matters.
        dictionary = dict(zip(index,zip(b,c)))
        return dictionary
    
def get_acc(dictionaryList): #input dictionaries, returns accession 
        accessions=[]
        for dictionary in dictionaryList:
            if isHuman(dictionary):
                accessions.append(dictionary['GBSeq_primary-accession'])
        return accessions
        
def get_seq(dictionaryList): #input dictionaries, returns sequences
        sequences=[]
        for dictionary in dictionaryList: 
            if isHuman(dictionary):  
                    sequences.append(dictionary["GBSeq_sequence"]) 
        return sequences
    
def get_acc_len_seq(dictionaryList): #input list of dictionaries, returns accession, length, and sequence
        name = []
        length=[]
        sequences=[]
        for dictionary in dictionaryList: #d iterates throughout each dictionary. each integer = 1 dictionary
            if isHuman(dictionary):  #i becomes a DICTIONARY and NOT an integer, which is why i'm not slicing. changed it to a number. this also means that i is no longer a dictionary, but an INT. 
                if "GBSeq_sequence" in dictionary: #if the TERM is in the dictionary
                    name.append(dictionary['GBSeq_primary-accession'])
                    length.append(dictionary['GBSeq_length']) # length of each protein 
                    sequences.append(dictionary["GBSeq_sequence"]) #then add that term to the list of sequences from the dict
        return dictCreate(name,length,sequences)

        
def remove_duplicates(l): #input list, returns list without duplicates
        return list(set(l))
    
def get_longest_isoform(seqList): #input list of sequences from get_seq, returns longest one. 
        longestIsoform=""
        remove_duplicates(seqList)
        for seq in seqList: 
            if len(seq)>len(longestIsoform):
                longestIsoform=seq
            else:
                pass
        return longestIsoform


def seqIO_fetch_proteins(accList):  #input accession list from get_acc function
        listprots=[]
        Entrez.email = 'rjliangiogenesis@gmail.com' 
        remove_duplicates(accList)
        for acc in accList:
            handle = Entrez.efetch(db='protein', 
                               rettype='gp', #9.6 of biopython documentation
                               retmode='text',  ##can't be xlm. has to be text. 
                               id=acc) 
            listprots.append(SeqIO.read(handle,"genbank"))
        return listprots
    
    
def seqIO_longest_isoform(listseq): #input seqIO list from fetch_all_proteinseq and returns longest isoform. 
    seqIO_long_iso=""
    for seq in listseq: 
        if len(seq.seq)> len(seqIO_long_iso): 
                seqIO_long_iso=seq.seq
        else:
                pass
        return seqIO_long_iso


In [39]:
#example of class in action: one method. we're calling the last guy. not good. 

five_protein_ID = get_IDs("MAP3K19",5)
fetched_document = fetch_details(five_protein_ID)
seq=get_seq(fetched_document)
len(get_longest_isoform(seq))

1328

In [40]:
#example 2 of class in action: method through seqIO

five_protein_IDs = get_IDs("MAP3K19",5)
fetched_documents = fetch_details(five_protein_IDs) 
doc_accessions = get_acc(fetched_documents)
seqIO_list = seqIO_fetch_proteins(doc_accessions)
longest_isoform = seqIO_longest_isoform(seqIO_list)
print(len(longest_isoform))

1328


In [8]:
seqIO_list[0].features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1328)), type='source'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1328)), type='gene'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1328)), type='Protein'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1328)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1139)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(78), ExactPosition(191)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(206), ExactPosition(1074)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(206), ExactPosition(1024)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(437), ExactPosition(438)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(499), ExactPosition(500)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(675), ExactPosition(676)), type='Region'),
 SeqFeature(FeatureLocation(ExactPosition(811), ExactPosition(812)), t

In [22]:
#using for-loops and making dicitonaries of longest seuqneces:
def get_family_number_articles(query):
    dictionary={}
    for num in range(25):
        if get_number_pubmed_queries(query+str(num))>0:
            dictionary.update({query+str(num):get_number_pubmed_queries(query+str(num))})
        else:
            pass
    return dictionary
    
def get_family_IDs(query,number): #input protein PREFIX and number of queries you want per family member, returns name:ID
    dictionary={}
    for num in range(25):
        dictionary.update({query+str(num):get_IDs(query+str(num),number)})
    return dictionary

def fetch_family_details(ID_dictionary): #input result from get_family_IDs, returns dictionary (name:dictionaries) of dictionaries(efetch)
    details={}
    for name,IDs in ID_dictionary.items():
        if len(IDs)>0:
            details.update({name:fetch_details(IDs)})
        else:
            pass
    return details
                           
def get_family_acc(article_dictionary): #input fetch_family_details result and returns dict of name:accessions list
    fam_accessions={}
    for name,articles in article_dictionary.items():
        fam_accessions.update({name:get_acc(articles)})
    return fam_accessions

    
def seqIO_fetch_family(accession_dictionary): #input get_family_acc and returns dict of name: seqIO list
    fam_sequences={}
    for name,accessions in accession_dictionary.items():
        fam_sequences.update({name:seqIO_fetch_proteins(accessions)})
    return fam_sequences
    
def seqIO_longest_family(seqIO_dictionary): #input seqIO_fetch_family results and retursn dict of name: longest seqIO
    fam_longseq={}
    for name,listseq in seqIO_dictionary.items():
        fam_longseq.update({name:seqIO_longest_isoform(listseq)})
    return fam_longseq
    

    
def get_family_seq(article_dictionary): #input fetch_family_details result and returns dict of name: sequence list
    fam_seq={}
    for name,articles in article_dictionary.items():
        fam_seq.update({name:get_seq(articles)})
    return fam_seq

def get_family_longest(seq_dictionary): #input get_family_seq results and returns dict of name: longest CDS
    fam_longest={}
    for name,seqList in seq_dictionary.items():
        fam_longest.update({name:get_longest_isoform(seqList)})
    return fam_longest
    

In [23]:
get_family_number_articles("MAP3K")

{'MAP3K1': 801,
 'MAP3K10': 49,
 'MAP3K11': 26,
 'MAP3K12': 21,
 'MAP3K13': 20,
 'MAP3K14': 57,
 'MAP3K15': 7,
 'MAP3K16': 1,
 'MAP3K17': 2,
 'MAP3K18': 1,
 'MAP3K19': 3,
 'MAP3K2': 88,
 'MAP3K20': 1,
 'MAP3K3': 117,
 'MAP3K4': 66,
 'MAP3K5': 514,
 'MAP3K6': 13,
 'MAP3K7': 117,
 'MAP3K8': 287,
 'MAP3K9': 34}

In [240]:
#example of method one. doesn't really work. 
family_IDs=get_family_IDs("MAP3K",5) #use a larger number to be super sure
family_details=fetch_family_details(family_IDs)
family_acc=get_family_acc(family_details)
family_seqIO_list=seqIO_fetch_family(family_acc)
family_longest_isoform=seqIO_longest_family(family_seqIO_list)
family_longest_isoform

for a,b in family_longest_isoform.items():
    print(a,len(b))


MAP3K1 66
MAP3K2 234
MAP3K3 90
MAP3K4 1607
MAP3K5 1374
MAP3K6 797
MAP3K7 579
MAP3K8 467
MAP3K9 1104
MAP3K10 954
MAP3K11 847
MAP3K12 567
MAP3K13 656
MAP3K14 947
MAP3K15 1313
MAP3K16 1001
MAP3K17 1235
MAP3K18 898
MAP3K19 1328
MAP3K20 800
MAP3K21 1036


In [38]:
#example of method two: works well. works better the more queries you start initially with. 
family_IDs=get_family_IDs("MAP3K",15) #use a larger number to be super sure
family_details=fetch_family_details(family_IDs)
family_seq=get_family_seq(family_details)
family_longest=get_family_longest(family_seq)
for a,b in family_longest.items():
    print(a,len(b))

MAP3K1 1512
MAP3K2 619
MAP3K3 657
MAP3K4 1608
MAP3K5 1374
MAP3K6 1288
MAP3K7 606
MAP3K8 467
MAP3K9 1118
MAP3K10 962
MAP3K11 847
MAP3K12 892
MAP3K13 966
MAP3K14 947
MAP3K15 1313
MAP3K16 1001
MAP3K17 1241
MAP3K18 907
MAP3K19 1328
MAP3K20 800
MAP3K21 1036


# HOW DO I JUST GET A SPECIFIC DOMAIN? FROM BLAST DATABASE? THIS WILL BE THE NEXT STEP.  OR MAYBE USE THE SPARCLE bio.sparcle http://mscbio2025.csb.pitt.edu/notes/biopython.slides.html#/15

USE this: https://warwick.ac.uk/fac/sci/moac/people/students/peter_cock/python/rpsblast/
--this should blast out the specific domains of each proteins sequence 

#can always call something on linux command line with python... play with? 

#try playing with RPSblast... download PSSM database
-use web api cdd_help.shtml 
#try playing with it. go on blast--.. standalone blast --> go to databases, download cdd_delta.targ.gz
cdd for delta blast?? 

--need to download RPSblast database... structure/cdd/cdd.shtml. 
cdd.tar.gz is all the pssms for blast search. 
building a custom datbase, read the readme file. outlines how to build a custom database. 

